## From REST to reasoning: ingest, index, and query with dlt and Cognee

Video: [https://www.youtube.com/watch?v=MNt_KK32gys](https://www.youtube.com/watch?v=MNt_KK32gys)

## Resources

[Slides](https://docs.google.com/presentation/d/1oHQilxEVqGGW4S2ctNEE0wHY2LgcjYLaRUziAoinsis/edit?usp=sharing)
[Colab Notebook](https://colab.research.google.com/drive/1vBA9OIGChcKjjg8r5hHduR0v3A5D6rmH?usp=sharing)
___
## Homework

### Question 1. dlt Version

In this homework, we will load the data from our FAQ to Qdrant

Let's install dlt with Qdrant support and Qdrant client:
```
pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"
```
What's the version of dlt that you installed? ```dlt==1.12.3```

In [1]:
pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip freeze

annotated-types==0.7.0
anyio==4.9.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==3.0.0
async-lru==2.0.5
attrs==25.3.0
babel==2.17.0
beautifulsoup4==4.13.3
bleach==6.2.0
certifi==2025.1.31
cffi==1.17.1
charset-normalizer==3.4.1
click==8.2.1
colorama==0.4.6
coloredlogs==15.0.1
comm==0.2.2
contourpy==1.3.1
cycler==0.12.1
debugpy==1.8.13
decorator==5.2.1
defusedxml==0.7.1
distro==1.9.0
dlt==1.12.3
elastic-transport==8.17.1
elasticsearch==9.0.2
executing==2.2.0
fastembed==0.7.1
fastjsonschema==2.21.1
filelock==3.13.1
flatbuffers==25.2.10
fonttools==4.56.0
fqdn==1.5.1
fsspec==2024.6.1
gitdb==4.0.12
GitPython==3.1.44
giturlparse==0.12.0
grpcio==1.73.0
h11==0.14.0
h2==4.2.0
hexbytes==1.3.1
hf-xet==1.1.3
hpack==4.1.0
httpcore==1.0.7
httpx==0.28.1
huggingface-hub==0.33.0
humanfriendly==10.0
humanize==4.12.3
hyperframe==6.1.0
idna==3.10
ipykernel==6.29.5
ipython==9.0.2
ipython_pygments_lexers==1.1.1
isoduration==20.11.0
jedi==0.19.2
Jinja2==3.1.6
jiter==0.10.0
joblib==

## dlt Resource
---

In [10]:
import dlt
import requests

@dlt.resource(write_disposition="replace", name="zoomcamp_dlt_data")
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

## Question 2. dlt pipeline
---
Now let's create a pipeline.

We need to define a destination for that. Let's use the ```qdrant``` one:

In [7]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In this case, we tell dlt (and Qdrant) to create a folder with our data, and the name for it will be ```db.qdrant```

Let's run it:

In [11]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Run started at 2025-07-07 19:51:45.424239+00:00 and COMPLETED in 5.07 seconds with 4 steps.
Step extract COMPLETED in 0.41 seconds.

Load package 1751917905.6700156 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.09 seconds.
Normalized data for the following tables:
- zoomcamp_dlt_data: 948 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1751917905.6700156 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 4.34 seconds.
Pipeline zoomcamp_pipeline load step completed in 4.33 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /workspaces/llm-workshop-datatalks/workshops/dlt/db.qdrant location to store data
Load package 1751917905.6700156 is LOADED and contains no failed jobs

Step run COMPLETED in 5.07 seconds.
Pipeline zoomcamp_pipeline load step completed in 4.33 seconds
1 load package(s) were 

How many rows were inserted into the zoomcamp_data collection?  ```948 rows```

Look for ```"Normalized data for the following tables:"``` in the trace output.

## Question 3. Embeddings
---
When inserting the data, an embedding model was used. Which one?  ```fast-bge-small-en```

You can find this out by inspecting the ```meta.json``` file created in the target folder.